In [1]:
import pandas as pd
import numpy as np
import os
from ts_data import TS_Data
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as F
from torchvision import transforms, utils
import torch.nn as nn
from CNN import CNN
from metrics import nll

In [3]:
fdir = "/home/warren/Documents/UGRA/AIM/csv_files"
def get_dataset(fdir, in_len,pred_len):
    l = os.listdir(fdir)
    datasets = [None]*len(l)
    for i in range(len(l)):
        temp = os.path.join(fdir,l[i])
        datasets[i] = TS_Data(temp, in_len,pred_len)
    dataset =  torch.utils.data.ConcatDataset(datasets)
    return dataset


In [4]:
inlen = 8
predlen=4
full_dataset = get_dataset(fdir,inlen, predlen)
train_len = int(.8* len(full_dataset))
test_len = len(full_dataset)-train_len
train_data, test_data = torch.utils.data.random_split(full_dataset, [train_len, test_len])
train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=256)

## Training 

In [4]:
#model = CNN()
#take out the z 

In [5]:
#model = nn.LSTM(2, 5,5, batch_first=True) #ensure that batch is first 
model = CNN(8,4)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nll()
optimizer = torch.optim.Adam(model.parameters(), lr = .0001)
epochs = 4


In [6]:
    #list of predictions is [batch size, steps - 8, ]    
def pred_to_samples(num_samples, pred):
        sx = torch.exp(pred[2]) #sx
        sy = torch.exp(pred[3]) #sy
        corr = torch.tanh(pred[4]) #corr
        sxsy = sx*sy
        cov = torch.FloatTensor([[sx*sx, corr*sxsy],[corr*sxsy, sy*sy]])
        means = torch.FloatTensor([pred[0],pred[1]])
        dist = torch.distributions.multivariate_normal.MultivariateNormal(means, cov)
        return dist.sample((num_samples,))
#no idea if I need to detach for these 

def ade_dist(pred,target):
        return (torch.sum(torch.square(pred-target))/pred.shape[1]).item() #assume the middle dimension


def fde_dist(pred, target):
        return (torch.sum(torch.square(pred[-1] - target[-1]))).item() #only compare the final location of pred to final location of tgt

In [7]:
def valid():
    print("valid")
    model.eval()
    with torch.no_grad():
        batch_sum_ade = 0
        batch_sum_fde=0
        for idx, (inputs, labels) in enumerate(test_dataloader): #batch size, steps, etc
            inputs = inputs.to(device)
            labels = labels.to(device)
            T = inputs.shape[0]
            N = inputs.shape[1]
            preds = model(inputs) #each step has a distribution. So take a sample of each step and then turn it into something cumulative

            for b in range(T): #iterate over batches
                curr = preds[b,:,:]
                #iterate over curr
                num_samples = 20
                num_steps = curr.shape[0]
                sample_list = []
                for i in range(num_steps):
                    sample_list.append(pred_to_samples(num_samples, curr[i]))
                    #now should have a list of samples for each step

                samples = torch.stack(sample_list,dim=1) #stack it along middle dim to get [num_samples, num_steps, (x,y)]
                #now cumsum it 
                abs_samples = samples.cumsum(dim=1) #dim =1 since it is the number of steps shape is [num_samples, steps, rel pos]
                # now take the minimum ade/fde of the absolute samples to it 
                fde_dists = []
                ade_dists = []
                temp_label = labels[b,:,:]
                temp_label = temp_label.cumsum(dim=0)
                for i in range(num_samples):
                    #calculate the metrics for each
                    fde_dists.append(fde_dist(abs_samples[i].to(device),temp_label))
                    ade_dists.append(ade_dist(abs_samples[i].to(device),temp_label))
                #now take the min
                min_fde = min(fde_dists)
                min_ade = min(ade_dists)
                batch_sum_ade+=min_ade
                batch_sum_fde+=min_fde
            
            
        batch_sum_ade/= len(test_dataloader)
        batch_sum_fde/= len(test_dataloader)
        valid_ade.append(batch_sum_ade)
        valid_fde.append(batch_sum_fde)

In [8]:
losses = []
valid_ade = []
valid_fde = []
for e in range(epochs):
    model.train()
    with torch.autograd.detect_anomaly():
        for idx, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds= model(inputs)
        #predictions are nan for some reason
            loss  = criterion(preds, labels) 
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),.001)
            optimizer.step()
            losses.append(loss.item())

    #Validation 
    valid()

    

<ipython-input-8-ee402afdc4d5>:6: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():
/home/warren/mlenv/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


valid


In [ ]:
    with torch.no_grad():
        batch_sum_ade = 0
        batch_sum_fde=0
        for idx, (inputs, labels) in enumerate(test_dataloader): #batch size, steps, etc
            inputs = inputs.to(device)
            labels = labels.to(device)
            T = inputs.shape[0]
            N = inputs.shape[1]
            preds = model(inputs) #each step has a distribution. So take a sample of each step and then turn it into something cumulative
            sample_list[i] = pred_to_samples(num_samples, curr[i])
                
                #iterate over curr
                num_samples = 20
                num_steps = curr.shape[0]
                sample_list = []*num_steps
                for i in range(num_steps):
                    sample_list[i] = pred_to_samples(num_samples, curr[i])
                    #now should have a list of samples for each step

                samples = torch.stack(samples,dim=1) #stack it along middle dim to get [num_samples, num_steps, (x,y)]
                #now cumsum it 
                abs_samples = samples.cumsum(dim=1) #dim =1 since it is the number of steps shape is [num_samples, steps, rel pos]
                # now take the minimum ade/fde of the absolute samples to it 
                fde_dists = []
                ade_dists = []
                for i in range(num_samples):
                    #calculate the metrics for each
                    fde_dists.append(fde_dist(abs_samples[i]))
                    ade_dists.append(ade_dist(abs_samples[i]))
                #now take the min
                min_fde = min(fde_dists)
                min_ade = min(ade_dists)
                batch_sum_ade+=min_ade
                batch_sum_fde+=min_fde
            
            
        batch_sum_ade/= len(test_dataloader)
        batch_sum_fde/= len(test_dataloader)
        print("done 1")
            

In [7]:
print(losses)

[3.1433467864990234, 3.042160749435425, 2.7600347995758057, 2.7089853286743164, 2.619258165359497, 2.7567954063415527, 2.5506069660186768, 2.5198793411254883, 2.678232192993164, 2.419008731842041, 2.6152989864349365, 2.4690394401550293, 2.4687514305114746, 2.3994381427764893, 2.3469181060791016, 2.336169719696045, 2.3863279819488525, 2.2950239181518555, 2.330838203430176, 2.239394187927246, 2.2979226112365723, 2.2310032844543457, 2.280668258666992, 2.1629772186279297, 2.1138134002685547, 2.1755824089050293, 2.104759693145752, 2.0242767333984375, 2.0752296447753906, 2.034060001373291, 2.0100159645080566, 1.9700212478637695, 2.029034376144409, 1.9620453119277954, 1.899348497390747, 1.864247441291809, 1.973174810409546, 1.8742237091064453, 1.817836046218872, 1.8262577056884766, 2.585263729095459, 1.801393985748291, 1.7678054571151733, 1.7637109756469727, 1.7639069557189941, 1.7116169929504395, 1.726269245147705, 1.6954078674316406, 1.649776816368103, 1.8043162822723389, 1.6419544219970703

# Issues
1. Had to change prediction length to the same as the input length for LSTM
2. Do i need to detach or is calling item() sufficient
3. The [batch_size, 8,5] means that each step has a distribution. So each x,y in a sequence of steps is from a different distribution and then make absolute
4. Making a CNN compatible with time series 

Assume take in 8 time steps, and predict 4 steps
1. Input into model, and model will return distribution parameters
2. Take a loss with 4 time steps and compare to distribution
3. 

1. Model predicts distribution, parameters, mux,muy sx,sy, rho
2. creat mean and cov matrix to make distribution. Torch multivariate_norm. tanh for coeff, exp for sx 
3. generate samples from distribution, 20 trajecctories . NOTE: each step has its own distribution. So for one trajectory, use 8 distributions
4. Convert samples from relative to absolute
4. Compare samples to the target one, take the minimum 




Issue with the prediction length, set it to the same as the input length. Changed the [:,1] to [...,1] etc

In [ ]:
for b in range(num_batches):
    curr = preds[b,:,:]
    #iterate over curr
    num_samples = 20
    num_steps = curr.shape[0]
    sample_list = []*num_steps
    for i in range(num_steps):
        sample_list[i] = pred_to_samples(num_samples, curr[i])
        #now should have a list of samples for each step
        
    samples = torch.stack(samples,dim=1) #stack it along middle dim to get [num_samples, num_steps, (x,y)]
    #now cumsum it 
    abs_samples = samples.cumsum(dim=1) #dim =1 since it is the number of steps shape is [num_samples, steps, rel pos]
    # now take the minimum ade/fde of the absolute samples to it 
    fde_dists = []
    ade_dists = []
    for i in range(num_samples):
        #calculate the metrics for each
        fde_dists.append(fde_dist(abs_samples[i]))
        ade_dists.append(ade_dist(abs_samples[i]))
    #now take the min
    min_fde = min(fde_dists)
    min_ade = min(ade_dists)
    
        

In [ ]:
a = torch.rand((5,4))

In [18]:
dist = torch.distributions.multivariate_normal.MultivariateNormal(torch.FloatTensor([0,0]), torch.eye(2))
samples = dist.sample((20))
samples.shape

torch.Size([20, 8, 2])

In [ ]:
samples[0]

In [23]:
s1 = pred_to_samples(20,torch.FloatTensor([1,0,1,1,0]))
s2 = pred_to_samples(20,torch.FloatTensor([1,0,1,1,0]))
print(s1[0])
print(s2[0])
a = [s1,s2]
print(len(a))
print(len(a[0]))
a = torch.stack(a, dim =1 )
print(a.shape)

tensor([-2.9822, -2.4117])
tensor([1.0272, 3.2640])
2
20
torch.Size([20, 2, 2])


In [27]:
a = torch.rand(5,2)
b = torch.rand(5,2)
print(a)
print(b)
torch.stack([a,b],dim=1).shape


tensor([[0.4471, 0.8284],
        [0.6522, 0.4346],
        [0.4059, 0.5153],
        [0.3018, 0.9224],
        [0.3225, 0.8794]])
tensor([[0.3184, 0.6030],
        [0.8760, 0.7945],
        [0.6209, 0.9229],
        [0.0354, 0.9280],
        [0.3968, 0.6559]])


torch.Size([5, 2, 2])

In [ ]:
a = (torch.rand((20,8,2))).cumsum(dim=1)
b = (torch.rand((20,8,2))).cumsum(dim=1)
ade_dist(a,b)

In [9]:
#replace L by H

a = torch.rand((4,2))
print(a)
a.cumsum(dim=0)

tensor([[0.8299, 0.1746],
        [0.7730, 0.9449],
        [0.9312, 0.6769],
        [0.4362, 0.0676]])


tensor([[0.8299, 0.1746],
        [1.6029, 1.1195],
        [2.5342, 1.7964],
        [2.9704, 1.8640]])